In [1]:
from PC_SAFT import flash, BPT_gekko
from Solve_ChemEQ import solve_ChemEQ_gekko
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [4]:
def get_x(CO2_loading, w_MEA):
    MW_MEA = 61.084
    MW_H2O = 18.02
    
    x_MEA_unloaded = w_MEA / (MW_MEA / MW_H2O + w_MEA * (1 - MW_MEA / MW_H2O))
    x_H2O_unloaded = 1 - x_MEA_unloaded
    
    n_MEA = 100 * x_MEA_unloaded
    n_H2O = 100 * x_H2O_unloaded

    n_CO2 = n_MEA * CO2_loading
    n_tot = n_MEA + n_H2O + n_CO2
    x_CO2, x_MEA, x_H2O = n_CO2/n_tot, n_MEA/n_tot, n_H2O/n_tot
    
    return x_CO2, x_MEA, x_H2O

print(get_x(.3, .3))

(0.03257503022901745, 0.10858343409672482, 0.8588415356742577)


In [5]:
def get_x(CO2_loading, w_MEA):

    MW_MEA = 61.08 / 1000
    MW_H2O = 18.02 / 1000

    x_MEA = (1 + CO2_loading + (MW_MEA / MW_H2O) * (1 - w_MEA) / w_MEA) ** -1
    x_CO2 = x_MEA*CO2_loading
    x_H2O = 1 - x_CO2 - x_MEA

    return x_CO2, x_MEA, x_H2O

print(get_x(.3, .3))

(0.03257686235281356, 0.10858954117604522, 0.8588335964711412)


In [3]:
def liquid_density(Tl, x, df_param):

    x_CO2, x_MEA, x_H2O = x

    MWs_l = np.array([44.01, 61.08, 18.02]) / 1000  # kg/mol

    MWT_l = sum([x[i] * MWs_l[i] for i in range(len(x))])

    a1, b1, c1 = [-5.35162e-7, -4.51417e-4, 1.19451]
    a2, b2, c2 = [-3.2484e-6, 0.00165, 0.793]

    V_MEA = MWs_l[1]*1000 / (a1 * Tl ** 2 + b1 * Tl + c1)  # mL/mol
    V_H2O = MWs_l[2]*1000 / (a2 * Tl ** 2 + b2 * Tl + c2)   # mL/mol

    # a, b, c, d, e = df_param['molar_volume'].values()
    a, b, c, d, e = 10.57920122, -2.020494157, 3.15067933, 192.0126008, -695.3848617

    V_CO2 = a + (b + c * x_MEA) * x_MEA * x_H2O + (d + e * x_MEA) * x_MEA * x_CO2

    V_l = V_CO2 * x_CO2 + x_MEA * V_MEA + x_H2O * V_H2O # Liquid Molar Volume (mL/mol)
    V_l = V_l*1e-6  # Liquid Molar Volume (mL/mol --> m3/mol)

    rho_mol_l = V_l**-1  # Liquid Molar Density (m3/mol --> mol/m3)
    rho_mass_l = rho_mol_l*MWT_l  # Liquid Mass Density (mol/m3 --> kg/m3)

    return rho_mol_l, rho_mass_l


In [11]:
df = pd.read_excel(r'C:\Users\Tanner\Documents\git\MEA\data\Jou_1995_30%.xlsx', sheet_name='T = 0')
P_CO2_data, α_data, P_data = df['P_CO2'].to_numpy()*1000, df['alpha'].to_numpy(), df['P'].to_numpy()*1000
P_interp = interp1d(α_data, P_data, kind='cubic')

In [14]:
Tl = 273.15 # K
α_range = np.linspace(α_data[0], α_data[-1], 21)
P_CO2_range = np.zeros(len(α_range))

for i, α in enumerate(α_range):

    x = get_x(α, .3)
    rho_mol_l, _ = liquid_density(Tl, x, [])
    Cl = [x[i] * rho_mol_l for i in range(len(x))]
    Cl_true = solve_ChemEQ_gekko(Cl, Tl)
    x_true = Cl_true / (sum(Cl_true)).astype('float')
    x = x_true[:3]


    m = np.array([2.0729, 3.0353, 1.9599])  # Number of segments
    σ = np.array([2.7852, 3.0435, 2.362])  # Temperature-Independent segment diameter σ_i (Aᵒ)
    ϵ_k = np.array([169.21, 277.174, 279.42]) # Depth of pair potential / Boltzmann constant (K)
    k_ij = np.array([[0.0, .16, .065],
                     [.16, 0.0, -.18],
                     [.065, -.18, 0.0]])
    κ_AB = np.array([0, .037470, .2039])
    ϵ_AB_k = np.array([0, 2586.3, 2059.28])

    yg = [.01, .00001, .98]
    Pg = P_interp(α)

    y, P = flash(x, yg, Tl, Pg, m, σ, ϵ_k, k_ij, flash_type='Bubble_T', κ_AB=κ_AB, ϵ_AB_k=ϵ_AB_k)

    P_CO2_range[i] = y[0]*P



C:\Users\Tanner\Documents\git\MEA\PC_SAFT\PC_SAFT.py:409: RuntimeWarning: invalid value encountered in log
  return np.exp(μ_res / self.kb / T - np.log(Z))


IndexError: list index out of range

In [ ]:
plt.plot(α_range, P_CO2_range, linestyle='--', color='black')
plt.plot(α_data, P_CO2_data, marker='x', linestyle='None')
plt.yscale('log')